In [ ]:
from urdfModifiers.core.linkModifier import LinkModifier
from urdfModifiers.core.jointModifier import JointModifier
from urdfModifiers import utils
from urdfpy import URDF
from urdfModifiers.geometry import * 
from urdfModifiers.geometry.geometry import Side
from urdfModifiers.utils import *
import idyntree.bindings as iDynTree
from urdfModifiers.geometry.geometry import Limb
import configparser

In [ ]:
def MoveRobotVisualization(index, delta):
    T = iDynTree.Transform()
    p = iDynTree.Position()
    for k in range(3):
        if(k== index):
            p.setVal(k,delta)
        else:
            p.setVal(k,0.0)
    
    T.setPosition(p)

    R = iDynTree.Rotation()
    for i in range(3): 
        for j in range(3):
            if(i == j):
                R.setVal(i,j,1.0)
            else:
                R.setVal(i,j,0.0)
    T.setRotation(R)
    return T

## Loading URDF

In [ ]:
urdf_path ="./models/stickBot/model.urdf"
output_file = "./models/stickBotModified.urdf"
dummy_file = 'no_gazebo_plugins.urdf'
robot, gazebo_plugin_text = utils.load_robot_and_gazebo_plugins(urdf_path,dummy_file)

## Defining the modifier for stickBot

In [ ]:
modifiers_mapper = {
    'r_upper_arm': [
        LinkModifier.from_name('r_upper_arm',robot, 0.022),
        JointModifier.from_name('r_elbow',robot, 0.0344)
    ],
    'r_forearm': [
        LinkModifier.from_name('r_forearm',robot, 0.03904),
        JointModifier.from_name('r_wrist_pitch',robot, 0.0506)
    ],
    'l_upper_arm': [
        LinkModifier.from_name('l_upper_arm',robot, 0.022),
        JointModifier.from_name('l_elbow',robot, 0.0344)
    ],
    'l_forearm': [
        LinkModifier.from_name('l_forearm',robot, 0.03904),
        JointModifier.from_name('l_wrist_pitch',robot, 0.0506)
    ],
    'r_hip_3': [
        LinkModifier.from_name('r_hip_3',robot, 0.058),
        JointModifier.from_name('r_hip_yaw',robot, 0.1451),
        JointModifier.from_name('r_knee',robot, 0.0536)
    ],
    'r_lower_leg': [
        LinkModifier.from_name('r_lower_leg',robot, -0.03),
        JointModifier.from_name('r_ankle_pitch',robot, -0.055989)
    ],
    'l_hip_3': [
        LinkModifier.from_name('l_hip_3',robot, 0.058),
        JointModifier.from_name('l_hip_yaw',robot, 0.1451),
        JointModifier.from_name('l_knee',robot, 0.0536)
    ],
    'l_lower_leg': [
        LinkModifier.from_name('l_lower_leg',robot, -0.03),
        JointModifier.from_name('l_ankle_pitch',robot, -0.055989)
    ],
    'root_link': [
        LinkModifier.from_name('root_link',robot, 0, Side.DEPTH, calculate_origin_from_dimensions = False),
        JointModifier.from_name('torso_pitch',robot, -0.078, flip_direction=False),
        JointModifier.from_name('r_hip_pitch',robot, 0.0494, take_half_length=True),
        JointModifier.from_name('l_hip_pitch',robot, 0.0494, take_half_length=True)
    ],
    'torso_1': [
        LinkModifier.from_name('torso_1',robot, 0, Side.DEPTH, calculate_origin_from_dimensions = False),
        JointModifier.from_name('torso_yaw',robot, -0.07113, flip_direction=False),
    ],
    'torso_2': [
        LinkModifier.from_name('torso_2',robot, 0, Side.DEPTH, calculate_origin_from_dimensions = False),
        JointModifier.from_name('torso_yaw',robot, -0.07113, flip_direction=False)
    ],
    'chest': [
        LinkModifier.from_name('chest',robot, 0, Side.DEPTH, calculate_origin_from_dimensions = False),
        JointModifier.from_name('r_shoulder_pitch',robot, 0.0554, take_half_length=True, flip_direction=False),
        JointModifier.from_name('l_shoulder_pitch',robot, 0.0554, take_half_length=True, flip_direction=False),
        JointModifier.from_name('neck_fixed_joint',robot, 0.0607, take_half_length=True, flip_direction=False)
    ]

}

## Loading the configuration file 

In [ ]:
config_file_path = "./config/conf.ini"
config = configparser.ConfigParser()
config.read(config_file_path)

## Reading modifications from urdf file

In [ ]:
sections_configuration_file = config.sections
for config_section in config.sections():
    modifications = utils.parse_modifications(config[config_section])            
    if config_section.upper() in Limb:
        selector = Limb[config_section.upper()]
    else: 
        selector = config_section
    if isinstance(selector, str):
        if selector in modifiers_mapper:
            elements_to_modify = modifiers_mapper[selector]
    elif (selector == Limb.RIGHT_ARM):
        elements_to_modify = modifiers_mapper["r_upper_arm"] + modifiers_mapper["r_forearm"]
    elif (selector == Limb.LEFT_ARM):
        elements_to_modify = modifiers_mapper["l_upper_arm"] + modifiers_mapper["l_forearm"]
    elif (selector == Limb.LEFT_LEG):
        elements_to_modify = modifiers_mapper["l_lower_leg"] + modifiers_mapper["l_hip_3"]
    elif (selector == Limb.RIGHT_LEG):
        elements_to_modify = modifiers_mapper["r_lower_leg"] + modifiers_mapper["r_hip_3"]
    elif (selector == Limb.ARMS):
        elements_to_modify = modifiers_mapper["r_upper_arm"] + modifiers_mapper["r_forearm"] + modifiers_mapper["l_upper_arm"] + modifiers_mapper["l_forearm"]
    elif (selector == Limb.LEGS):
        elements_to_modify = modifiers_mapper["l_lower_leg"] + modifiers_mapper["l_hip_3"] + modifiers_mapper["r_lower_leg"] + modifiers_mapper["r_hip_3"]
    elif (selector == Limb.TORSO):
        elements_to_modify = modifiers_mapper["root_link"] + modifiers_mapper["torso_1"] + modifiers_mapper["torso_2"] + modifiers_mapper["chest"]
    elif (selector == Limb.ALL):
        all_modifiers = []
        for i in modifiers_mapper:
            all_modifiers += modifiers_mapper[i]
        elements_to_modify = all_modifiers
    elif (selector == Limb.NONE):
        elements_to_modify = []
    else:
        elements_to_modify = []


## Apply Modifications

In [ ]:
for element_to_modify in elements_to_modify:
    element_to_modify.modify(modifications)

## Writing to urdf

In [ ]:
utils.write_urdf_to_file(robot, output_file, gazebo_plugin_text)
print("urdf saved in ", output_file)

## Visualize Output 

In [ ]:
## Loading Urdf in iDynTree
mdlLoaderOriginal = iDynTree.ModelLoader()
mdlLoaderModified = iDynTree.ModelLoader()
mdlLoaderOriginal.loadReducedModelFromFile(urdf_path,[] , 'root_link')
mdlLoaderModified.loadReducedModelFromFile(output_file,[] ,'root_link')

## Setting up the visualizer
viz = iDynTree.Visualizer() 
vizOpt = iDynTree.VisualizerOptions()
viz.init()
viz.setColorPalette("meshcat")
env = viz.enviroment()
env.setElementVisibility('floor_grid',False)
env.setElementVisibility('world_frame',False)
cam = viz.camera()
cam.setPosition(iDynTree.Position(2.5,0,1.2))
viz.camera().animator().enableMouseControl(True)

## Adding the model
viz.addModel(mdlLoaderOriginal.model(), 'originalModel')
viz.addModel(mdlLoaderModified.model(),'modifiedModel')
viz.modelViz('originalModel').setModelColor(iDynTree.ColorViz(iDynTree.Vector4_FromPython([1,0.2,0.2,0.2])))

## Move the model to improve visibility 
T=MoveRobotVisualization(1, 1.0)
viz.modelViz('originalModel').setPositions(T,iDynTree.VectorDynSize())

## Visualizing the models 
while(viz.run()):
    viz.draw()  